# 실시간 유추 서비스 만들기

학습시킨 예측 모델은 클라이언트가 새 데이터에서 예측 정보를 가져오는 데 사용할 수 있는 실시간 서비스로 배포할 수 있습니다.

## 작업 영역에 연결합니다.

이 Notebook의 작업을 시작하려면 먼저 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 모델 학습 및 등록

이제 모델 학습과 등록을 진행하겠습니다.

In [ ]:
from azureml.core import Experiment
from azureml.core import Model
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 작업 영역에서 Azure ML 실험 만들기
experiment = Experiment(workspace=ws, name="mslearn-train-diabetes")
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# 당뇨병 데이터 세트 로드
print("Loading Data...")
diabetes = pd.read_csv('data/diabetes.csv')

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 의사 결정 트리 모델 학습 진행
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# 학습된 모델 저장
model_file = 'diabetes_model.pkl'
joblib.dump(value=model, filename=model_file)
run.upload_file(name = 'outputs/' + model_file, path_or_stream = './' + model_file)

# 실행 완료
run.complete()

# 모델 등록
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Inline Training'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

print('Model trained and registered.')

## 모델을 웹 서비스로 배포

당뇨 환자일 가능성을 기준으로 환자를 분류하는 기계 학습 모델의 학습과 등록을 완료했습니다. 프로덕션 환경에서는 당뇨 의심 환자만 당뇨 임상 시험 대상으로 지정해야 하는 수술 등에 이 모델을 사용할 수 있습니다. 이 시나리오를 지원하려는 경우 웹 서비스로 모델을 배포합니다.

먼저 작업 영역에 등록한 모델을 확인해 보겠습니다.

In [ ]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

이제 배포할 모델을 가져옵니다. 기본적으로는 모델 이름을 지정하면 최신 버전이 반환됩니다.

In [ ]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

이 모델을 호스트하는 웹 서비스를 만들려면 몇 가지 코드와 구성 파일이 필요합니다. 먼저 이러한 항목을 저장할 폴더를 만들겠습니다.

In [ ]:
import os

folder_name = 'diabetes_service'

# 웹 서비스 파일용 폴더 만들기
experiment_folder = './' + folder_name
os.makedirs(experiment_folder, exist_ok=True)

print(folder_name, 'folder created.')

# 채점 스크립트용 경로 설정
script_file = os.path.join(experiment_folder,"score_diabetes.py")

모델을 배포하는 웹 서비스에는 입력 데이터를 로드하고, 작업 영역에서 모델을 가져오고, 예측을 생성/반환하기 위한 특정 Python 코드가 필요합니다. 웹 서비스에 배포할 *입력 스크립트*(대개 *채점 스크립트*라고 함)에 이 코드를 저장할 것입니다.

In [ ]:
%%writefile $script_file
import json
import joblib
import numpy as np
from azureml.core.model import Model

# 서비스를 로드하면 호출됨
def init():
    global model
    # 배포된 모델 파일의 경로를 가져와서 로드
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# 요청이 수신되면 호출됨
def run(raw_data):
    # 입력 데이터를 numpy 배열로 가져오기
    data = np.array(json.loads(raw_data)['data'])
    # 모델에서 예측 가져오기
    predictions = model.predict(data)
    # 각 예측(0 또는 1)에 해당하는 클래스 이름 가져오기
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # 예측을 JSON으로 반환
    return json.dumps(predicted_classes)

웹 서비스는 컨테이너에서 호스트되며, 이 컨테이너는 초기화 시에 필수 Python 종속성을 설치해야 합니다. 여기서는 채점 코드에 **scikit-learn**이 필요하므로 환경에 scikit-learn을 설치하도록 컨테이너 호스트에 명령하는 .yml 파일을 만들겠습니다.

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies 

# 모델의 종속성 추가(AzureML 기본값은 이미 포함되어 있음)
myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn')

# 환경 구성을 .yml 파일로 저장
env_file = os.path.join(experiment_folder,"diabetes_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# .yml 파일 인쇄
with open(env_file,"r") as f:
    print(f.read())

이제 배포를 진행할 수 있습니다. 컨테이너에 **diabetes-service** 서비스를 배포할 것입니다. 배포 프로세스에는 다음 단계가 포함됩니다.

1. 모델을 로드하고 사용하는 데 필요한 채점 및 환경 파일을 포함하는 유추 구성을 정의합니다.
2. 서비스를 호스트할 실행 환경을 정의하는 배포 구성을 정의합니다. 여기서 실행 환경은 Azure Container Instance입니다.
3. 모델을 웹 서비스로 배포합니다.
4. 배포된 서비스의 상태를 확인합니다.

> **추가 정보**: 모델 배포 및 대상 실행 환경용 옵션에 대한 자세한 내용은 [설명서](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)를 참조하세요.

배포에서는 컨테이너 이미지를 만드는 프로세스를 먼저 실행한 다음 해당 이미지를 기반으로 웹 서비스를 만드는 프로세스를 실행하므로 시간이 다소 걸릴 수 있습니다. 배포가 정상적으로 완료되면 **정상** 상태가 표시됩니다.

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# 채점 환경 구성
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetes-service"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

배포가 정상적으로 진행되었다면 **정상** 상태를 확인할 수 있습니다. 정상 상태가 표시되지 않으면 다음 코드를 사용하여 서비스 로그를 가져와 문제 해결 시에 참조할 수 있습니다.

In [ ]:
print(service.get_logs())

# 서비스를 변경 및 재배포해야 하는 경우 다음 코드를 사용하여 비정상 서비스를 삭제해야 할 수 있습니다.
#service.delete()

[Azure Machine Learning Studio](https://ml.azure.com)의 작업 영역을 살펴보고 작업 영역에서 배포된 서비스가 표시되는 **엔드포인트** 페이지를 확인합니다.

다음 코드를 실행하여 작업 영역에서 웹 서비스 이름을 검색할 수도 있습니다.

In [ ]:
for webservice_name in ws.webservices:
    print(webservice_name)

## 웹 서비스 사용

배포한 서비스는 클라이언트 애플리케이션에서 사용할 수 있습니다.

In [ ]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# 배열을 JSON 문서의 직렬화 가능 목록으로 변환
input_json = json.dumps({"data": x_new})

# 웹 서비스를 호출하여 입력 데이터 전달(웹 서비스는 이진 형식 데이터도 수락함)
predictions = service.run(input_data = input_json)

# 예측된 클래스(첫 번째/유일한 클래스) 가져오기
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

여러 환자를 관찰한 정보를 서비스로 전송한 후 각 환자에 대한 예측을 다시 가져올 수도 있습니다.

In [ ]:
import json

# 이번에는 입력이 기능 배열 2개로 구성된 배열입니다.
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# 배열 하나 이상을 JSON 문서의 직렬화 가능 목록으로 변환
input_json = json.dumps({"data": x_new})

# 웹 서비스를 호출하여 입력 데이터 전달
predictions = service.run(input_data = input_json)

# 예측된 클래스 가져오기
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

위의 코드는 Azure Machine Learning SDK를 사용하여 컨테이너화된 웹 서비스에 연결한 다음 이 서비스를 사용하여 당뇨병 분류 모델에서 예측을 생성합니다. 프로덕션 환경에서는 Azure Machine Learning SDK를 사용하지 않으며 웹 서비스로의 HTTP 요청만 수행하는 비즈니스 애플리케이션이 모델을 사용할 가능성이 높습니다.

이러한 애플리케이션이 요청을 제출해야 하는 URL을 확인해 보겠습니다.

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

엔드포인트 URI가 확인되면 애플리케이션은 HTTP 요청을 수행하여 JSON 형식으로 환자 데이터를 전송한 다음 예측된 클래스를 다시 수신할 수 있습니다.

In [ ]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# 배열을 JSON 문서의 직렬화 가능 목록으로 변환
input_json = json.dumps({"data": x_new})

# 콘텐츠 형식 설정
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

인증이 필요하지 않은 ACI(Azure Container Instance) 서비스로 웹 서비스를 배포했습니다. 개발 및 테스트 시에는 인증을 사용하지 않아도 되지만, 프로덕션 환경에서는 AKS(Azure Kubernetes Service) 클러스터에 서비스를 배포하고 토큰 기반 인증을 사용하도록 설정하는 것이 좋습니다. 이렇게 하려면 REST 요청에 **인증** 헤더를 포함해야 합니다.

## 서비스 삭제

더 이상 필요하지 않은 서비스는 불필요한 요금이 발생하지 않도록 삭제해야 합니다.

In [ ]:
service.delete()
print ('Service deleted.')

모델을 서비스로 게시하는 과정에 대한 자세한 내용은 [설명서](https://docs.microsoft.com/azure/machine-learning/how-to-deploy-and-where)를 참조하세요.